In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers
from tqdm import trange

In [2]:
df = pd.read_csv('data/single_step_df_ints_2022-06-03_encoded.csv', header=0)

In [3]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,p.already_seen,n.already_seen,w.already_seen,t.already_seen,g.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2069,<start>,d,1.000000,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2070,d,i,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2071,i,k,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2072,k,x,0.000000,1.500000,2.000000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [5]:
df['target'].unique()
df['target'].value_counts()

k    278
p    249
f    172
g    168
d    164
c    152
s    142
b    110
i    106
w     74
a     74
o     73
r     60
u     56
q     43
v     30
h     28
e     22
t     22
z     21
m     13
n      9
x      7
y      1
Name: target, dtype: int64

In [6]:
index_to_drop = df['target'].loc[df['target'] == 'y'].index[0]
index_to_drop

1945

In [7]:
# remove row with target that only occurs once in whole dataset

df_new = df.drop(index=index_to_drop, axis=0)
df_new.reset_index(inplace=True, drop=True)

In [8]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [9]:
df_new[1438:]['target'].value_counts()

d    137
k    116
i    104
p     61
u     56
q     43
b     32
v     30
f     15
r     13
s      8
g      8
x      5
c      4
a      2
h      1
Name: target, dtype: int64

In [10]:
index_to_drop_cooking = df_new[1438:]['target'].loc[df_new[1438:]['target'] == 'h'].index[0]
index_to_drop_cooking

df_cooking = df_new.drop(index=index_to_drop_cooking, axis=0)
df_cooking.reset_index(inplace=True, drop=True)

In [11]:
df_cooking[1437:]

,input,target,start_coords.x,start_coords.y,start_coords.z,p.already_seen,n.already_seen,w.already_seen,t.already_seen,g.already_seen,...,v.food_k,v.strong_k,v.mid_k,coordinates_y.x,coordinates_y.y,coordinates_y.z,y.containment,y.food_k,y.strong_k,y.mid_k
1437,<start>,i,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1438,i,k,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1439,k,d,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1440,d,p,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1441,<start>,d,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2067,<start>,d,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2068,d,i,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2069,i,k,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2070,k,x,0.0,1.5,2.0,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [12]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_cooking[1437:], df_cooking[1437:]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [13]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [14]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_cooking['target'].unique()

In [15]:
len(labels)

23

In [16]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [17]:
batch_size = 128

In [18]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_32633/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}
2022-06-07 10:23:39.112052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 10:23:39.135290: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 10:23:39.135410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-07 10:23:39.135789: I tensorflow/core/platform/c

In [19]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [20]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [21]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [22]:
all_inputs, encoded_features = create_input_data(df)

In [23]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [24]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [25]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [60]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [26]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
4/4 [==============================] - 6s 673ms/step - loss: 3.1096 - accuracy: 0.0991 - val_loss: 2.4950 - val_accuracy: 0.2188
Epoch 2/100
4/4 [==============================] - 0s 47ms/step - loss: 2.6024 - accuracy: 0.1779 - val_loss: 2.2945 - val_accuracy: 0.2292
Epoch 3/100
4/4 [==============================] - 0s 46ms/step - loss: 2.3844 - accuracy: 0.2162 - val_loss: 2.1877 - val_accuracy: 0.2188
Epoch 4/100
4/4 [==============================] - 0s 46ms/step - loss: 2.2270 - accuracy: 0.1982 - val_loss: 2.1020 - val_accuracy: 0.2604
Epoch 5/100
4/4 [==============================] - 0s 46ms/step - loss: 2.1899 - accuracy: 0.2455 - val_loss: 2.0133 - val_accuracy: 0.3021
Epoch 6/100
4/4 [==============================] - 0s 46ms/step - loss: 2.0983 - accuracy: 0.2365 - val_loss: 1.9515 - val_accuracy: 0.2917
Epoch 7/100
4/4 [==============================] - 0s 48ms/step - loss: 1.9882 - accuracy: 0.2883 - val_loss: 1.9121 - val_accuracy: 0.2917
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 46ms/step - loss: 1.0286 - accuracy: 0.6351 - val_loss: 1.4511 - val_accuracy: 0.5104
Epoch 60/100
4/4 [==============================] - 0s 47ms/step - loss: 1.0379 - accuracy: 0.6306 - val_loss: 1.4355 - val_accuracy: 0.5417
Epoch 61/100
4/4 [==============================] - 0s 46ms/step - loss: 1.0322 - accuracy: 0.6171 - val_loss: 1.4246 - val_accuracy: 0.5521
Epoch 62/100
4/4 [==============================] - 0s 46ms/step - loss: 0.9849 - accuracy: 0.6532 - val_loss: 1.4245 - val_accuracy: 0.5208
Epoch 63/100
4/4 [==============================] - 0s 46ms/step - loss: 1.0195 - accuracy: 0.6509 - val_loss: 1.4323 - val_accuracy: 0.5104
Epoch 64/100
4/4 [==============================] - 0s 46ms/step - loss: 0.9949 - accuracy: 0.6667 - val_loss: 1.4370 - val_accuracy: 0.5312
Epoch 65/100
4/4 [==============================] - 0s 47ms/step - loss: 0.9808 - accuracy: 0.6622 - val_loss: 1.4498 - val_accuracy: 0.5104
Epoch 66/100
4/4 [========

In [27]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 74ms/step - loss: 1.4572 - accuracy: 0.5158
Accuracy 0.5157894492149353


In [28]:
model.save('models/next_obj_classifier_cooking_2022-06-07')

2022-06-07 10:25:09.100783: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_cooking_2022-06-07/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_cooking_2022-06-07/assets


In [29]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_cooking_2022-06-07/')

In [ ]:
# test prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [30]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [30]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in trange(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [32]:
nr_of_sequences_ts = len(df[:1438][df[:1438]['input'] == '<start>'])
nr_of_sequences_ts

187

In [39]:
errors = get_prequential_error(df, reloaded_model, labels, nr_of_sequences)

100%|████████████████████████████████████████████████████████████████| 2073/2073 [11:56<00:00,  2.89it/s]


In [40]:
summed_error = [sum(error) for error in errors[:-1]]

In [41]:
np.median(summed_error)

4.0

In [42]:
with open('results/nn_spatialinfo_prequential_summed_fitted_on_cooking_results_all_2022-06-07.txt', 'w') as file:
    file.write(str(summed_error))

In [43]:
np.mean(summed_error)

5.074433656957929

In [44]:
len(summed_error)

309